In [18]:
from google.cloud import bigquery
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import datetime
from google.cloud import bigquery
import pytz
from google.oauth2 import service_account
import numpy as np
import requests
from datetime import datetime as dt
from datetime import date


In [19]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'.\creds\opendatadbt-f2e674527eb6.json'
client = bigquery.Client()

credentials = ServiceAccountCredentials.from_json_keyfile_name(r'.\creds\opendatadbt-f2e674527eb6.json') # Your json file here



#Initial load chicago_table_id = "opendatadbt.311.chicago311" chicago311 = requests.get("https://data.cityofchicago.org/resource/v6vf-nfxy.json? limit=10000& where=created_date >= '2021-03-18T00:00:00.000'" ) chicago311json = chicago311.json() chicago_df = pd.DataFrame(chicago311json) chicago_df = chicago_df[['created_date','closed_date','owner_department','sr_type','zip_code','street_address','city','status','latitude','longitude']] chicago_df

In [21]:
#Incremental load
chicago_table_id = "opendatadbt.311.chicago311"
current_datetime = dt.today().strftime("%Y-%m-%dT%H:%M:%S.000")
query_string = """
SELECT 
max(created_date), max(closed_date)
FROM `opendatadbt.311.chicago311`
limit 10
"""

chicagomaxopendatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chicagomaxopendatedate = str(chicagomaxopendatedataframe['f0_'][0])

chicagomaxclosedatedataframe = (
    client.query(query_string)
    .result()
    .to_dataframe()
)
chicagomaxclosedate = str(chicagomaxclosedatedataframe['f1_'][0])


In [22]:
chicagomaxclosedate

'2021-03-25T08:15:06.000'

In [23]:
chicago311 = requests.get("https://data.cityofchicago.org/resource/v6vf-nfxy.json?$limit=50000&$where=created_date > '" + chicagomaxopendatedate +"' OR closed_date between '" +chicagomaxclosedate+ "' AND '" + current_datetime + "'") 
chicago311json = chicago311.json()
cols = ['sr_number','created_date','closed_date','owner_department','sr_type','zip_code','street_address','city','status','latitude','longitude']
chicago_df = pd.DataFrame(chicago311json, columns = cols)

chicago_df

,sr_number,created_date,closed_date,owner_department,sr_type,zip_code,street_address,city,status,latitude,longitude
0,SR20-05665985,2020-12-21T07:30:01.000,2021-03-25T08:23:50.000,Streets and Sanitation,Garbage Cart Maintenance,60641,2849 N KOSTNER AVE,NaN,Completed,41.933094889,-87.736571819
1,SR21-00311379,2021-02-24T11:48:58.000,2021-03-25T08:15:30.000,DOB - Buildings,No Building Permit and Construction Violation,60643,10629 S HALE AVE,NaN,Completed,41.700057382,-87.668861397
2,SR20-05674257,2020-12-22T09:37:52.000,2021-03-25T09:00:30.000,Streets and Sanitation,Garbage Cart Maintenance,60630,4910 N KRUGER AVE,NaN,Completed,41.970292122,-87.745820447
3,SR20-05684057,2020-12-23T14:01:10.000,2021-03-25T08:33:07.000,Streets and Sanitation,Garbage Cart Maintenance,60609,4842 S WOLCOTT AVE,NaN,Completed,41.805486306,-87.672447974
4,SR21-00449880,2021-03-18T16:35:23.000,2021-03-25T08:18:53.000,Streets and Sanitation,Abandoned Vehicle Complaint,60651,839 N LAWNDALE AVE,NaN,Completed,41.896611172,-87.718759759
...,...,...,...,...,...,...,...,...,...,...,...
1286,SR21-00106097,2021-01-19T16:50:13.000,2021-03-25T08:41:06.000,Streets and Sanitation,Garbage Cart Maintenance,60655,2610 W 102ND ST,NaN,Completed,41.708397224,-87.687161805
1287,SR21-00287100,2021-02-20T04:25:22.000,2021-03-25T08:56:07.000,BACP - Business Affairs and Consumer Protection,Cab Feedback,60631,5800 N CUMBERLAND AVE,NaN,Completed,41.983436622,-87.837537332
1288,SR21-00290429,2021-02-20T20:08:06.000,2021-03-25T09:04:45.000,BACP - Business Affairs and Consumer Protection,Public Vehicle/Valet Complaint,60626,1404 W ESTES AVE,NaN,Completed,42.011562108,-87.666305348
1289,SR21-00291857,2021-02-21T09:23:39.000,2021-03-25T09:07:34.000,BACP - Business Affairs and Consumer Protection,Cab Feedback,60611,247 E CHESTNUT ST,NaN,Completed,41.89845181,-87.620204287


In [24]:
chicago_table_id = "opendatadbt.311.chicago311"
dataset_ref = client.dataset('311')
table_ref = dataset_ref.table('chicago311')
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)

client.load_table_from_dataframe(
    chicago_df, table_ref, job_config=job_config
).result()  # Make an API request.
